In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install retina-face

In [3]:
import cv2
def capFrame(videoPath, savePath,famename):
    cap = cv2.VideoCapture(videoPath)
    numFrame = 0
    imagenum = int(cap.get(7)/60)
    # print(imagenum)
    while True:
        if imagenum == 0:
             break
        if cap.grab():
            numFrame += 1
            # 每60桢截取一个图片
            if numFrame % 60 == 1:
                #retrieve 解码并返回一个桢
                flag, frame = cap.retrieve()
                cv2_imshow(frame)
                newPath = savePath + str(famename)+ str("_")+str(int(imagenum)) +".jpg"
                # print(newPath)
                cv2.imencode('.jpg', frame)[1].tofile(newPath)
                imagenum -= 1
       
# capFrame(r"C:\Users\fulian\Desktop\code and exp\manipulated_sequences\Face2Face\c23\videos\023_923.mp4",r"C:\Users\fulian\Desktop\image")


In [ ]:
import os
from google.colab.patches import cv2_imshow
path_fake= "/content/drive/MyDrive/Deepfakes/c23/videos"  #待读取的文件夹
path_list_fake=os.listdir(path_fake)
for filename in path_list_fake:
            m = str(os.path.splitext(filename)[0])
            capFrame(os.path.join(path_fake,filename),"/content/drive/MyDrive/picture/",m)

In [ ]:
import os
from google.colab.patches import cv2_imshow
path= "/content/drive/MyDrive/videos_original"  #待读取的文件夹
path_list=os.listdir(path)
for filename in path_list:
            m = str(os.path.splitext(filename)[0])
            capFrame(os.path.join(path,filename),"/content/drive/MyDrive/orignal_unextract/",m)

In [ ]:
import matplotlib.pyplot as plt
from retinaface import RetinaFace
import torch
import os
import cv2
import torchvision
import numpy as np

path='/content/drive/MyDrive/orignal_unextract'
path_list=os.listdir(path)
i = 0

for filename in path_list:
   m = str(os.path.splitext(filename)[0])
   faces = RetinaFace.extract_faces(img_path = os.path.join(path,filename), align = True)
   for face in faces:
      plt.imshow(face)
      savepath = '/content/drive/MyDrive/original_face_img/'+str(m)+'face'+'.jpg'
      plt.savefig(savepath)
      img = cv2.imread(savepath)
      img = torch.tensor(img)
      img = torch.unsqueeze(img,0)
      # print(img.shape)
      # img = torch.flatten(img,1,3)
      img = img.permute(0,3,1,2)
      #print(img.shape)
      if i == 0:
        org_set = img
      elif i != 0:
        org_set = torch.cat((org_set,img),0)
      i += 1
      plt.show()

  

In [ ]:
import matplotlib.pyplot as plt
from retinaface import RetinaFace
import torch
import os
import cv2
path='/content/drive/MyDrive/picture'
path_list=os.listdir(path)
j = 0

for filename in path_list:
   m = str(os.path.splitext(filename)[0])
   faces = RetinaFace.extract_faces(img_path = os.path.join(path,filename), align = True)
   for face in faces:
      plt.imshow(face)
      savepath = '/content/drive/MyDrive/fake_face_img/'+str(m)+'face'+'.jpg'
      plt.savefig(savepath)
      img = cv2.imread(savepath)
      img = torch.tensor(img)
      img = torch.unsqueeze(img,0)
      # print(img.shape)
      # img = torch.flatten(img,1,3)
      img = img.permute(0,3,1,2)
      print(img.shape)
      # print(img)
      if j == 0:
        fake_set = img
      elif j != 0:
        fake_set = torch.cat((fake_set,img),0)
      j += 1
      plt.show()

  
  

In [ ]:
import torch.utils.data as Data
print(org_set.shape)
print(i)
length_ori = i
# print(j)

label_o = torch.ones(i)
label_f = torch.zeros(j)

dataset_t = torch.cat((fake_set,org_set),0)  

print(dataset_t.shape)
label = torch.cat((label_o,label_f),0) 
print(label.shape)
dataset = Data.TensorDataset(dataset_t, label)
#分割
validation_split = .2
random_seed= 42
dataset_size = len(dataset)
indices = list(range(dataset_size))
np.random.seed(random_seed)
np.random.shuffle(indices)
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = Data.SubsetRandomSampler(train_indices)
valid_sampler = Data.SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=5, 
                                   sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=5,
                                   sampler=valid_sampler)
# list(validation_loader)#

In [12]:
"""

Author: Andreas Rössler
"""
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo

pretrained_settings = {
    'xception': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/xception-b5690688.pth',
            'input_space': 'RGB',
            'input_size': [3,240,320],
            'input_range': [0, 1],
            'mean': [0.5, 0.5, 0.5],
            'std': [0.5, 0.5, 0.5],
            'num_classes': 1000,
            'scale': 0.8975  # The resize parameter of the validation transform should be 333, and make sure to center crop at 299x299
        }
    }
}

class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False):
        super(SeparableConv2d, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size,
                               stride, padding, dilation, groups=in_channels, bias=bias)
        self.pointwise = nn.Conv2d(
            in_channels, out_channels, 1, 1, 0, 1, 1, bias=bias)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x

class RegressionMap(nn.Module):
    def __init__(self, c_in):
        super(RegressionMap, self).__init__()
        self.c = SeparableConv2d(c_in, 1, 3, stride=1, padding=1, bias=False)
        self.s = nn.Sigmoid()

    def forward(self, x):
        mask = self.c(x)
        mask = self.s(mask)
        return mask

class Block(nn.Module):
    def __init__(self, in_filters, out_filters, reps, strides=1, start_with_relu=True, grow_first=True):
        super(Block, self).__init__()

        if out_filters != in_filters or strides != 1:
            self.skip = nn.Conv2d(in_filters, out_filters,
                                  1, stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip = None

        self.relu = nn.ReLU(inplace=False)
        rep = []

        filters = in_filters
        if grow_first:   # whether the number of filters grows first
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters, out_filters,
                                       3, stride=1, padding=1, bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(self.relu)
            rep.append(SeparableConv2d(filters, filters,
                                       3, stride=1, padding=1, bias=False))
            rep.append(nn.BatchNorm2d(filters))

        if not grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters, out_filters,
                                       3, stride=1, padding=1, bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3, strides, 1))
        self.rep = nn.Sequential(*rep)

    def forward(self, inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp

        x += skip
        return x


class Xception(nn.Module):
    """
    Xception optimized for the ImageNet dataset, as specified in
    https://arxiv.org/pdf/1610.02357.pdf
    """

    def __init__(self, num_classes=1000, inc=3):
        """ Constructor
        Args:
            num_classes: number of classes
        """
        super(Xception, self).__init__()
        self.num_classes = num_classes

        # Entry flow
        self.conv1 = nn.Conv2d(inc, 32, 3, 2, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=False)

        self.conv2 = nn.Conv2d(32, 64, 3, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        # do relu here

        self.block1 = Block(
            64, 128, 2, 2, start_with_relu=False, grow_first=True)
        self.block2 = Block(
            128, 256, 2, 2, start_with_relu=True, grow_first=True)
        self.block3 = Block(
            256, 728, 2, 2, start_with_relu=True, grow_first=True)

        # middle flow
        self.block4 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)
        self.block5 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)
        self.block6 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)
        self.block7 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)

        self.block8 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)
        self.block9 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)
        self.block10 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)
        self.block11 = Block(
            728, 728, 3, 1, start_with_relu=True, grow_first=True)

        # Exit flow
        self.block12 = Block(
            728, 1024, 2, 2, start_with_relu=True, grow_first=False)

        self.conv3 = SeparableConv2d(1024, 1536, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(1536)

        # do relu here
        self.conv4 = SeparableConv2d(1536, 2048, 3, 1, 1)
        self.bn4 = nn.BatchNorm2d(2048)

        self.fc = nn.Linear(2048, num_classes)
        self.type_fc = nn.Linear(2048, 5)
        self.mag_fc = nn.Linear(2048, 1)
        self.map = RegressionMap(728)
        self.pecent= 1.0 / 1.5
    def fea_part1_0(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)  

        return x

    def fea_part1_1(self, x):  
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x) 

        return x
    
    def fea_part1(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)     
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x) 

        return x
    
    def fea_part2(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)

        return x

    def fea_part3(self, x):
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)

        return x

    def fea_part4(self, x):
        x = self.block8(x)
        x = self.block9(x)
        x = self.block10(x)
        x = self.block11(x)
        x = self.block12(x)

        return x

    def fea_part5(self, x):
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.conv4(x)
        x = self.bn4(x)

        return x
     
    def features(self, input):
        x = self.fea_part1(input)    

        x = self.fea_part2(x)
        x = self.fea_part3(x)
        x = self.fea_part4(x)

        x = self.fea_part5(x)
        return x

    #def classifier(self, features):
    def classifier(self, x):
        x = self.fea_part4(x)
        x = self.fea_part5(x)
        x = self.relu(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        out = self.last_linear(x)
        type = self.type_fc(x)
        mag = self.mag_fc(x)
        return out, x, type, mag

    def estimateMap(self, x):
        map = self.map(x)
        return map

    #def forward(self, input):
    def forward(self, x):
        x = self.fea_part1(x)
        x = self.fea_part2(x)
        x = self.fea_part3(x)
        out, fea, type, mag = self.classifier(x)
        map = self.estimateMap(x)
        return out, fea, map, type, mag


def xception(num_classes=1000, pretrained='imagenet', inc=3):
    model = Xception(num_classes=num_classes, inc=inc)
    if pretrained:
        settings = pretrained_settings['xception'][pretrained]
        assert num_classes == settings['num_classes'], \
            "num_classes should be {}, but is {}".format(
                settings['num_classes'], num_classes)

        model = Xception(num_classes=num_classes)
        model.load_state_dict(model_zoo.load_url(settings['url']))

        model.input_space = settings['input_space']
        model.input_size = settings['input_size']
        model.input_range = settings['input_range']
        model.mean = settings['mean']
        model.std = settings['std']

    # TODO: ugly
    model.last_linear = model.fc
    del model.fc
    return model



class RPModel(nn.Module):
    def __init__(self,modelchoice,num_out_classes=2,dropout=0.0):
        super(RPModel, self).__init__()
        self.modelchoice = modelchoice

        if modelchoice == 'resnet50':
            self.rmodel = torchvision.models.resnet50(pretrained=True)
        elif modelchoice == 'resnet18':
            self.rmodel = torchvision.models.resnet18(pretrained=True)
        elif modelchoice == 'resnext':
            self.rmodel = torchvision.models.resnext50_32x4d(pretrained=True)
        elif modelchoice == 'inceptionv3':
            self.rmodel = torchvision.models.inception_v3(pretrained=True)
        # elif modelchoice == 'efficientB5':
        #     self.rmodel = EfficientNet.from_pretrained('efficientnet-b5')
        # elif modelchoice == 'efficientB7':
        #     self.rmodel = EfficientNet.from_pretrained('efficientnet-b7')
        else: 
            raise ValueError('No matching model...')


        if modelchoice == 'efficientB5' or modelchoice == 'efficientB7':
            num_ftrs = self.rmodel._fc.in_features
            if not dropout:
                self.rmodel._fc = nn.Linear(num_ftrs, num_out_classes)
            else:
                self.rmodel._fc = nn.Sequential(
                    nn.Dropout(p=dropout),
                    nn.Linear(num_ftrs, num_out_classes)
                )
        elif modelchoice == 'inceptionv3':
            num_ftrs = self.rmodel.AuxLogits.fc.in_features
            self.rmodel.AuxLogits.fc = nn.Linear(num_ftrs, num_out_classes)
            num_ftrs = self.rmodel.fc.in_features
            self.rmodel.fc = nn.Linear(num_ftrs,num_out_classes)
        else:
            num_ftrs = self.rmodel.fc.in_features
            if not dropout:
                self.rmodel.fc = nn.Linear(num_ftrs, num_out_classes)
            else:
                self.rmodel.fc = nn.Sequential(
                    nn.Dropout(p=dropout),
                    nn.Linear(num_ftrs, num_out_classes)
                )


    def features(self, input):
        return None # logits 

    def classifier(self, features):
        return None, None

    def forward(self, input):
        out = self.rmodel(input)

        return out, None


class TransferModel(nn.Module):
    """
    Simple transfer learning model that takes an imagenet pretrained model with
    a fc layer as base model and retrains a new fc layer for num_out_classes
    """

    def __init__(self, modelchoice, num_out_classes=2, dropout=0.0, 
    weight_norm=False, return_fea=False, inc=3):
        super(TransferModel, self).__init__()
        self.modelchoice = modelchoice
        self.return_fea = return_fea

        if modelchoice == 'xception':

            def return_pytorch04_xception(pretrained=True):
                # Raises warning "src not broadcastable to dst" but thats fine
                model = xception(pretrained=False)
                if pretrained:
                    # Load model in torch 0.4+
                    model.fc = model.last_linear
                    del model.last_linear
                    state_dict = torch.load(
                        '/content/drive/MyDrive/weights/xception-b5690688.pth')
                    print('Loaded pretrained model (ImageNet)....')
                    for name, weights in state_dict.items():
                        if 'pointwise' in name:
                            state_dict[name] = weights.unsqueeze(
                                -1).unsqueeze(-1)
                    model.load_state_dict(state_dict, strict=False)
                    model.last_linear = model.fc
                    del model.fc
                return model

            self.model = return_pytorch04_xception()
            # Replace fc
            num_ftrs = self.model.last_linear.in_features
            if not dropout:
                if weight_norm:
                    print('Using Weight_Norm')
                    self.model.last_linear = nn.utils.weight_norm(
                        nn.Linear(num_ftrs, num_out_classes), name='weight')
                self.model.last_linear = nn.Linear(num_ftrs, num_out_classes)
            else:
                print('Using dropout', dropout)
                if weight_norm:
                    print('Using Weight_Norm')
                    self.model.last_linear = nn.Sequential(
                        nn.Dropout(p=dropout),
                        nn.utils.weight_norm(
                            nn.Linear(num_ftrs, num_out_classes), name='weight')
                    )

                self.model.last_linear = nn.Sequential(
                    nn.Dropout(p=dropout),
                    nn.Linear(num_ftrs, num_out_classes)
                )
            
            if inc != 3:
                self.model.conv1 = nn.Conv2d(inc, 32, 3, 2, 0, bias=False)                
                nn.init.xavier_normal_(self.model.conv1.weight.data, gain=0.02)

        elif modelchoice == 'resnet50' or modelchoice == 'resnet18' \
            or modelchoice == 'resnext' or modelchoice == 'inceptionv3' \
            or modelchoice == 'efficientB5'or modelchoice == 'efficientB7':
            
            self.model = RPModel(modelchoice,num_out_classes,dropout)
            
        else:
            raise Exception('Choose valid model, e.g. resnet50')

    def set_trainable_up_to(self, boolean=False, layername="Conv2d_4a_3x3"):
        """
        Freezes all layers below a specific layer and sets the following layers
        to true if boolean else only the fully connected final layer
        :param boolean:
        :param layername: depends on lib, for inception e.g. Conv2d_4a_3x3
        :return:
        """
        # Stage-1: freeze all the layers
        if layername is None:
            for i, param in self.model.named_parameters():
                param.requires_grad = True
                return
        else:
            for i, param in self.model.named_parameters():
                param.requires_grad = False
        if boolean:
            # Make all layers following the layername layer trainable
            ct = []
            found = False
            for name, child in self.model.named_children():
                if layername in ct:
                    found = True
                    for params in child.parameters():
                        params.requires_grad = True
                ct.append(name)
            if not found:
                raise NotImplementedError('Layer not found, cant finetune!'.format(
                    layername))
        else:
            if self.modelchoice == 'xception':
                # Make fc trainable
                for param in self.model.last_linear.parameters():
                    param.requires_grad = True

            elif self.modelchoice in ['efficientB5','efficientB7']:
                # Make fc trainable
                for param in self.model._fc.parameters():
                    param.requires_grad = True           
            else:
                # Make fc trainable
                for param in self.model.fc.parameters():
                    param.requires_grad = True

    def forward(self, x):
        #out, x = self.model(x)
        out, x, map, type, mag = self.model(x)
        if self.return_fea:
            return out, x, map, type, mag
        else:
            return out


    def features(self, x):
        x = self.model.features(x)
        return x


class RawXception(nn.Module):
    """
    Untrained Xception Model
    """

    def __init__(self, num_out_classes=2, inc=3, dropout=0.0):
        super(RawXception, self).__init__()

        self.model = xception(pretrained=None, inc=inc)
        # Replace fc
        num_ftrs = self.model.last_linear.in_features
        if not dropout:
            self.model.last_linear = nn.Linear(num_ftrs, num_out_classes)
        else:
            print('Using dropout', dropout)
            self.model.last_linear = nn.Sequential(
                nn.Dropout(p=dropout),
                nn.Linear(num_ftrs, num_out_classes)
            )

    def forward(self, x):
        x = self.model(x)
        return x

    def features(self, x):
        x = self.model.features(x)
        return x

    def classifier(self, x):
        x = self.model.classifier(x)
        return x


def model_selection(modelname, num_out_classes,
                    dropout=None):
    """
    :param modelname:
    :return: model, image size, pretraining<yes/no>, input_list
    """
    if modelname == 'xception':
        return TransferModel(modelchoice='xception',
                             num_out_classes=num_out_classes)
    elif modelname == 'resnet18':
        return TransferModel(modelchoice='resnet18', dropout=dropout,
                             num_out_classes=num_out_classes), \
            224, True, ['image'], None
    else:
        raise NotImplementedError(modelname)


In [ ]:
Xmodel = model_selection('xception',2)
print(Xmodel)
Xmodel = Xmodel.eval()

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as Data





def get_acc(outputs, labels):
    """计算acc"""
    _, predict = torch.max(outputs.data, 1)
    total_num = labels.shape[0]*1.0
    correct_num = (labels == predict).sum().item()
    acc = correct_num / total_num

    return acc





model = model_selection('xception',2).eval()

# 优化器
optim = torch.optim.Adam(model.parameters(), lr=0.0001)

# 损失
loss_fun = nn.CrossEntropyLoss()

for e in range(10):
    epoch_loss = 0
    epoch_acc = 0
    for i, (x, y) in enumerate(train_loader):
        optim.zero_grad()
        out = model(x.float())
        
        y = y.type(torch.LongTensor)
        loss = loss_fun(out, y)

        loss.backward()
        optim.step()

        epoch_loss += loss.data
        epoch_acc += get_acc(out, y)
        

    if e % 5 == 0:
        print('epoch: %d, loss: %f, acc: %f' % (e, epoch_loss / 50, epoch_acc / 50))



In [15]:
torch.save(model.state_dict(),'/content/drive/MyDrive/classify_model.pkl')

In [16]:
from math import floor
import numpy as np
import cv2 as cv
import random

# 注释为： 攻击类型 参数的意义 （取值范围） 有较明显观感差距的取值

def GB(f,m): #高斯滤波 f为图片数组，m为高斯数组大小 5
    return cv.GaussianBlur(f,(m,m),1)

def AF(f,m): #中值滤波 f为图片数组，m为中值数组大小 5
    return cv.medianBlur(f, m)

def MF(f,m): #均值滤波 f为图片数组，m为均值数组大小 5
    return cv.blur(f, (m,m))

def JP(f,qual): #JPEG压缩 f为图片数组，qual为压缩质量，0-100 越高质量越好 50
    f=f.astype(np.uint8)
    cv.imwrite("temp.jpg",f,[int(cv.IMWRITE_JPEG_QUALITY),qual])
    return cv.imread("temp.jpg")

def CA(f,m): #对比度调节 f为图片数组，m为调节的对比度 -1-0-3  0.5
    img=f.astype(np.float64)
    out=img
    MH=128
    out=MH+(out-MH)*(1+m)
    out[out<0]=0
    out[out>255]=255
    return out.astype(np.uint8)

def CB(f,m): #亮度调节 f为图片数组，m为调节的亮度 百分比 -1-0-1 负数为降低亮度 正数增加  0.5
    out=f.astype(np.float64)
    out=out+m*255
    out[out<0]=0
    out[out>255]=255
    return out.astype(np.uint8)

def SC(f,m): #尺寸缩放 f为图片数组，m为调整的比例 百分比 0-1-正无穷 小于1为缩小，大于为放大 0.5
    l=f.shape[0]
    w=f.shape[1]
    l=int(l*m)
    w=int(w*m)
    g=cv.resize(f,(w,l))
    return g

def DIBR_right(f,m): #向右偏移 f为图片数组，m为偏移的比例 0-1 0.05
    m=m/4
    l=f.shape[0]
    w=f.shape[1]
    wm=int(m*w)
    g=f
    a=np.round(np.linspace(1,wm,l//2)).astype(np.int32)
    k=0
    if np.ndim(f)==3:
        temp=np.zeros((3,))
        for i in range(l-l//2,l):
            g[i,a[k]:w,:]=g[i,0:w-a[k],:]
            g[i,0:a[k],:]=np.zeros((a[k],3))
            k+=1
    else:
        for i in range(l-l//2,l):
            g[i,a[k]:w]=g[i,0:w-a[k]]
            g[i,0:a[k]]=np.zeros((a[k]))
            k+=1
    return g

def DIBR_left(f,m): #向左偏移 f为图片数组，m为偏移的比例 0-1 0.05
    m=m/4
    l=f.shape[0]
    w=f.shape[1]
    wm=int(m*w)
    g=f
    a=np.round(np.linspace(1,wm,l//2)).astype(np.int32)
    k=0
    if np.ndim(f)==3:
        temp=np.zeros((3,))
        for i in range(l-l//2,l):
            g[i,0:w-a[k],:]=g[i,a[k]:w,:]
            g[i,w-a[k]:w,:]=np.zeros((a[k],3))
            k+=1
    else:
        for i in range(l-l//2,l):
            g[i,0:w-a[k]]=g[i,a[k]:w]
            g[i,w-a[k]:w]=np.zeros((a[k]))
            k+=1
    return g

def Rotation(f,angle): #旋转 f为图片数组，m为旋转角度 360°制 90
    l=f.shape[0]
    w=f.shape[1]
    M=cv.getRotationMatrix2D((w//2,l//2),angle,1.0)
    return cv.warpAffine(f,M,(w,l)) 

def Crop(f,m): #边缘剪切 f为图片数组，m为剪切的比例，0-1  0.2
    if np.ndim(f)==3:
        l,w,h=f.shape
        m=m/2
        sl=floor(l*m)
        el=l-floor(l*m)
        sw=floor(w*m)
        ew=w-floor(w*m)
        newf=np.zeros((l,w,h),dtype=np.uint8)
        newf[sl:el,sw:ew]=f[sl:el,sw:ew]
    else:
        l,w=f.shape
        m=m/2
        sl=floor(l*m)
        el=l-floor(l*m)
        sw=floor(w*m)
        ew=w-floor(w*m)
        newf=np.zeros((l,w),dtype=np.uint8)
        newf[sl:el,sw:ew]=f[sl:el,sw:ew]
    return newf

def SN(f,perce): #椒盐噪声 f为图片数组，perce为噪声密度 0-1 0.01
    NoiseImg=f
    NoiseNum=int(perce*f.shape[0]*f.shape[1])
    for _ in range(NoiseNum):
        randX=random.randint(0,f.shape[0]-1)
        randY=random.randint(0,f.shape[1]-1)
        if np.ndim(f)==3:
            for i in range(3):
                if random.randint(0,1)<=0.5:
                    NoiseImg[randX,randY,i]=0
                else:
                    NoiseImg[randX,randY,i]=255
        else:
            if random.randint(0,1)<=0.5:
                NoiseImg[randX,randY]=0
            else:
                NoiseImg[randX,randY]=255
    return NoiseImg

def GN(image,var): #高斯噪声 image为图片数组，var为高斯滤波方差 0.01
    sigma=var**0.5
    image=np.array(image/255, dtype=float)
    noise = np.random.normal(0, sigma, image.shape)
    out = image + noise 
    out[out<0]=0
    out[out>1]=1
    out = np.uint8(out*255)
    return out
    
    

In [17]:
from symbol import parameters
import numpy as np
import cv2 as cv


def add_noise(picpath,savepath,famename):
  
    pic=cv.imread(picpath) #被攻击图片路径
    list = ["高斯模糊","均值滤波","中值滤波","亮度变化","对比度变化","边缘剪切","中心旋转","高斯噪声","椒盐噪声","尺寸缩放","DIBR左偏","DIBR右偏"]
    path= savepath+str(famename)+'noise'+'.jpg'#存放攻击后图片路径
    num = random.randint(0,11)
    # num = 9
   
    type=list[num] #攻击类型 请参考下面的注释使用
      
    
    # pic=cv.cvtColor(pic,cv.COLOR_BGR2GRAY)

    # 注释为： 攻击类型 参数的意义 （取值范围） 有较明显观感差距的取值
    if type == "高斯模糊": 
        #高斯滤波 f为图片数组，m为高斯数组大小 5
        parameter=5
        pic = GB(pic, parameter)

    elif type == "均值滤波":
        #均值滤波 f为图片数组，m为均值数组大小 5
        parameter=5
        pic = MF(pic, parameter)
    elif type == "中值滤波":
        #中值滤波 f为图片数组，m为中值数组大小 5
        parameter=5
        pic = AF(pic, parameter)
    elif type == "亮度变化":
        #亮度调节 f为图片数组，m为调节的亮度 百分比 -1-0-1 负数为降低亮度 正数增加  0.5
        parameter=0.5
        pic = CB(pic, parameter)
    elif type == "对比度变化":
        #对比度调节 f为图片数组，m为调节的对比度 -1-0-3  0.5
        parameter=0.5
        pic = CA(pic, parameter)
    elif type == "边缘剪切":
        #边缘剪切(实际效果为被剪切的部分变黑,图片大小未变) f为图片数组，m为剪切的比例，0-1  0.2
        parameter=0.2
        pic = Crop(pic, parameter)
    elif type == "中心旋转":
        #旋转 f为图片数组，m为旋转角度 360°制 90
        parameter=90
        pic = Rotation(pic, parameter)
    elif type == "高斯噪声":
        #高斯噪声 image为图片数组，var为高斯滤波方差 0.01
        parameter=0.01
        pic = GN(pic, parameter)
    elif type == "椒盐噪声":
        #椒盐噪声 f为图片数组，perce为噪声密度 0-1 0.01
        parameter=0.01
        pic = SN(pic, parameter)
    elif type == "尺寸缩放":
        #尺寸缩放 f为图片数组，m为调整的比例 百分比 0-1-正无穷 小于1为缩小，大于为放大 0.5
        parameter=0.5
        pic = SC(pic, parameter)
        #这里的DIBR仅为简易实现的左右像素偏移，慎用
    elif type == "DIBR左偏": 
        #向左偏移 f为图片数组，m为偏移的比例 0-1 0.05
        parameter=0.05
        pic = DIBR_left(pic, parameter)
    elif type == "DIBR右偏":
        #向右偏移 f为图片数组，m为偏移的比例 0-1 0.05
        parameter=0.05
        pic = DIBR_right(pic, parameter)

    #保存图片文件
    cv.imwrite(path, pic)
    img = cv2.imread(path)
    return img,list[num]



In [ ]:
import os
import cv2
import torch
from google.colab.patches import cv2_imshow
path_fake= "/content/drive/MyDrive/original_face_img"  #待读取的文件夹
path_list_fake=os.listdir(path_fake)
t1 = 0

for filename in path_list_fake:
            m = str(os.path.splitext(filename)[0])
            img,n = add_noise(os.path.join(path_fake,filename),'/content/drive/MyDrive/original_noise/',m)
            img = torch.tensor(img)
            
            img = torch.unsqueeze(img,0)
            img = img.permute(0,3,1,2)
            
            # print(n)
            # print(img.shape)
            
            # print(b.shape)
            if n == '尺寸缩放':
              for i in range(3):
               # print(img[0,i,:,:].shape)
               piece = img[0,i,:,:]
               piece = piece.numpy() 
               # print(img)
               temp = cv2.copyMakeBorder(piece,72,72,108,108,cv2.BORDER_REPLICATE)
               temp = torch.tensor(temp)
               temp = torch.unsqueeze(temp,0)
               if i == 0:
                 pic = temp
               elif i != 0:
                #  print(pic.shape)
                 # print(temp.shape)
                 pic = torch.cat((pic,temp),0)
               i += 1
              img = torch.unsqueeze(pic,0)
              # print(img.shape)
            if t1 == 0:
              org_set_noise = img
            elif t1 != 0:
              org_set_noise = torch.cat((org_set_noise,img),0)
            t1 += 1
print(org_set_noise.shape)
            

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
path_fake= "/content/drive/MyDrive/fake_face_img"  #待读取的文件夹
path_list_fake=os.listdir(path_fake)
t2 = 0
for filename in path_list_fake:
            m = str(os.path.splitext(filename)[0])
            img,n = add_noise(os.path.join(path_fake,filename),'/content/drive/MyDrive/fake_noise/',m)
            img = torch.tensor(img)
            
            img = torch.unsqueeze(img,0)
            img = img.permute(0,3,1,2)
            if n == '尺寸缩放':
              for i in range(3):
               # print(img[0,i,:,:].shape)
               piece = img[0,i,:,:]
               piece = piece.numpy() 
               # print(img)
               temp = cv2.copyMakeBorder(piece,72,72,108,108,cv2.BORDER_REPLICATE)
               temp = torch.tensor(temp)
               temp = torch.unsqueeze(temp,0)
               if i == 0:
                 pic = temp
               elif i != 0:
                #  print(pic.shape)
                 # print(temp.shape)
                 pic = torch.cat((pic,temp),0)
               i += 1
              img = torch.unsqueeze(pic,0)
              # print(img.shape)
            if t2 == 0:
             fake_set_noise = img
            elif t2 != 0:
              fake_set_noise = torch.cat((fake_set_noise,img),0)
            t2 += 1
print(fake_set_noise.shape)